In [94]:
import os
import phoenix
import phoenix as px
import pandas as pd
import openai
import re
import nest_asyncio
nest_asyncio.apply()

In [95]:
# 將 PHOENIX_COLLECTOR_ENDPOINT 指向本機實際 IP
#os.environ["PHOENIX_COLLECTOR_ENDPOINT"] = "http://10.234.14.175:6006" #學校
os.environ["PHOENIX_COLLECTOR_ENDPOINT"] = "http://192.168.1.122:6006" #家
client = px.Client()
dataset = client.get_dataset(
    name="only3-dataset",
    version_id="RGF0YXNldFZlcnNpb246MTE="
)

In [96]:
#df = dataset.as_dataframe()
#print(df.columns)
#df["input"] = df["input"].apply(lambda x: x['question'])
#df["output"] = df["output"].apply(lambda x: x['respond'])
#df["metadata"] = df["metadata"].apply(lambda x: x['reference'])


In [97]:
export_records = [] #把資料存到資料表中

client = openai.OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    base_url=os.getenv("OPENAI_API_BASE")
)

async def explain(example, prompt_template: str, model: str) -> str:
    formatted_prompt_template = prompt_template.format(input_message=example.input["question"],output_message=example.output["respond"],reference_message=example.metadata["reference"])
    res = client.chat.completions.create(
        model="o1",
        temperature=0,
        messages=[
            {"role": "user", "content": formatted_prompt_template},
        ],
    )
    assert res.choices
    return res.choices[0].message.content

In [98]:
import textwrap
from functools import partial

template = """
你是一個判斷事實真偽的專家，請根據{reference_message}的內容，判斷{output_message}對於{input_message}的回答中，是否有與事實不符的地方。

以條列解釋你如何判斷為"事實"或"幻覺"
"""

task = partial(explain, prompt_template=template, model="o1")
# print(dataset.examples)
test_example = dataset[0]
print(textwrap.fill(await task(test_example), width=100))

根據您提供的文本「艾菲爾鐵塔位於法國巴黎。它建於 1889 年，作為 1889 年世界博覽會的入口拱門。」，我判斷「艾菲爾鐵塔位於法國巴黎」為**事實**。  以下是我判斷的條列解釋：  *
**文本直接陳述：** 文本開宗明義地指出「艾菲爾鐵塔位於法國巴黎」，這是一個明確的地理位置聲明。 *   **普遍認知：**
艾菲爾鐵塔位於巴黎是全球公認的事實，且有大量證據支持（例如地圖、旅遊資訊、歷史記錄等）。 *   **文本內容一致性：**
文本的後續描述（建造年份、博覽會）都與艾菲爾鐵塔在巴黎的歷史背景相符，沒有任何矛盾之處。 *   **缺乏幻覺指標：** 文本中沒有任何不合理、矛盾或缺乏證據支持的資訊，因此可以排除是幻覺。
因此，基於文本內容和普遍認知，我確定「艾菲爾鐵塔位於法國巴黎」是**事實**，而對於艾菲爾鐵塔位於哪裡？的回答中**沒有與事實不符的地方**。


In [99]:
def judge(input, output, metadata) -> str:
    input_message = input["question"]
    output_message = output
    reference_message = metadata["reference"]
    print(input_message)
    print(output_message)
    print(reference_message)
    
    MY_CUSTOM_TEMPLATE = f'''

[BEGIN DATA]
[Question]: {input_message}
[Response]: {output_message}
[Reference]: {reference_message}
[END DATA]

# 角色
你是一個判斷事實真偽的專家，請根據[Reference]的內容，判斷[Response]對於[Question]的回答中，是否有與事實不符的地方。

# 輸出格式
"事實"或"幻覺"  

'''
    
    res = client.chat.completions.create(
        model="o1",
        temperature=0,
        messages=[
            {"role": "user", "content": MY_CUSTOM_TEMPLATE},
        ]
    )
    content = res.choices[0].message.content
    content = content.strip().lower()
    return content

EVALUATORS = [judge]

In [100]:
from phoenix.experiments import run_experiment
experiment= run_experiment(
    dataset,
    task,
    experiment_name="explanation",
    experiment_metadata={"vendor": "openai", "model": "o1"},
    evaluators=EVALUATORS,
)

🧪 Experiment started.
📺 View dataset experiments: http://192.168.1.122:6006/datasets/RGF0YXNldDo5/experiments
🔗 View this experiment: http://192.168.1.122:6006/datasets/RGF0YXNldDo5/compare?experimentId=RXhwZXJpbWVudDo2Mw==


running tasks |          | 0/3 (0.0%) | ⏳ 00:00<? | ?it/s

/opt/conda/lib/python3.10/site-packages/phoenix/utilities/client.py:51: UserWarning: ⚠️⚠️ The Phoenix server (8.12.0) and client (9.5.0) versions are severely mismatched. Upgrade  either the client or server to ensure API compatibility ⚠️⚠️
  warnings.warn(


✅ Task runs completed.
🧠 Evaluation started.


running experiment evaluations |          | 0/3 (0.0%) | ⏳ 00:00<? | ?it/s

艾菲爾鐵塔位於哪裡？ 
根據提供的內容「艾菲爾鐵塔位於法國巴黎。它建於 1889 年，作為 1889 年世界博覽會的入口拱門。」，我判斷「艾菲爾鐵塔位於法國巴黎」是**事實**。

以下是我的判斷依據：

*   **直接陳述：** 文本直接陳述「艾菲爾鐵塔位於法國巴黎」，沒有任何含糊或否定語氣。
*   **廣為人知的事實：** 艾菲爾鐵塔位於巴黎是全球廣為人知的地理事實，可以透過多種可靠來源（例如書籍、地圖、官方網站）驗證。
*   **文本一致性：** 文本中關於艾菲爾鐵塔建造年份和目的的描述與歷史事實相符，進一步強化了其地理位置的真實性。

因此，我沒有發現任何與事實不符的地方。
艾菲爾鐵塔位於法國巴黎。它建於 1889 年，作為 1889 年世界博覽會的入口拱門。 
世界上最長的河流是什麼？ 
好的，我來分析你提供的資訊，並判斷其中是否有與事實不符之處。

**我的判斷：** 該資訊部分正確，部分有待商榷。

**以下是我的條列式分析：**

1.  **尼羅河向北流經非洲東北部，從布隆迪最遠的源頭流入地中海：** 這是**事實**。尼羅河的流向大致正確，源頭的描述也大致正確（雖然關於尼羅河源頭的爭議一直存在，但布隆迪的卡蓋拉河通常被認為是尼羅河最遠的源頭之一）。

2.  **尼羅河全長約 6,650 公里（4,132 英里）：** 這是**幻覺** (不完全正確)。 雖然6,650公里是一個常見的尼羅河長度估計值，但最新的測量結果表明尼羅河的長度略短於亞馬遜河。目前普遍認為亞馬遜河才是世界上最長的河流。

3.  **尼羅河是世界上最長的河流：** 這是**幻覺** (不正確)。根據目前最新的科學測量，亞馬遜河才是世界上最長的河流，長度約為6,992公里（4,345英里）。

**總結：**

你提供的資訊在描述尼羅河的流向和大致長度上是正確的，但在斷言尼羅河是世界上最長的河流這一點上是錯誤的。目前科學界普遍認為亞馬遜河才是世界上最長的河流。
尼羅河是世界上最長的河流。它向北流經非洲東北部，從布隆迪最遠的源頭流入地中海，全長約 6,650 公里（4,132 英里）。 


/opt/conda/lib/python3.10/site-packages/phoenix/utilities/client.py:51: UserWarning: ⚠️⚠️ The Phoenix server (8.12.0) and client (9.5.0) versions are severely mismatched. Upgrade  either the client or server to ensure API compatibility ⚠️⚠️
  warnings.warn(


人體有幾塊骨頭？ 
根據你提供的文本：「人體有206塊骨頭。這些骨骼提供結構、保護器官、固定肌肉並儲存鈣。」 以及你的問題，我來判斷「成年人的身體通常有 256 塊骨頭」是否為事實或幻覺：

*   **文本基礎：** 文本明確指出人體有206塊骨頭。
*   **對比分析：** 文本提供的資訊與「256塊骨頭」的說法不符。
*   **判斷結果：** 因此，「成年人的身體通常有 256 塊骨頭」是**幻覺**。

**條列解釋：**

1.  **來源可信度：** 你提供的文本本身是關於人體骨骼的描述，可視為一個可信的來源。
2.  **直接矛盾：** 文本直接聲明人體有206塊骨頭，而你的問題斷言有256塊，兩者直接矛盾。
3.  **科學常識：** 206塊骨頭是醫學和生物學上公認的成年人骨骼數量。
4.  **缺乏佐證：** 問題中沒有提供任何證據支持「256塊骨頭」的說法。

因此，基於以上分析，我判斷「成年人的身體通常有 256 塊骨頭」是與事實不符的，屬於幻覺。
人體有206塊骨頭。這些骨骼提供結構、保護器官、固定肌肉並儲存鈣。 

🔗 View this experiment: http://192.168.1.122:6006/datasets/RGF0YXNldDo5/compare?experimentId=RXhwZXJpbWVudDo2Mw==

Experiment Summary (06/15/25 06:16 PM +0000)
--------------------------------------------
  evaluator  n  n_labels        top_2_labels
0     judge  3         3  {'事實': 2, '幻覺': 1}

Tasks Summary (06/15/25 06:16 PM +0000)
---------------------------------------
   n_examples  n_runs  n_errors
0           3       3         0
